# Neural Network Dev Notebook

This notebook is for development of a suitable neural network for training on the training set. Here, I use the dev set and tune hyperparameters and model structure for best performance. 

In [1]:
import keras
import glob
import os
from PIL import Image
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tnrange, tqdm_notebook
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten, Lambda, BatchNormalization
from keras.layers import Conv1D, ZeroPadding1D, MaxPooling1D, GlobalMaxPooling1D
from keras import backend as K
from keras.optimizers import Adam
import matplotlib
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from utils import load_json, make_logger
import logging
import itertools
import pickle
import difflib

params = load_json('params.json')
logger = make_logger('development', 'log/development.log')

Using TensorFlow backend.


## Load the dev set and images

Creates a dict relating artist MBIDs to spectrogram images, represented as 128x128x1 arrays of floats. Also load the previously saved database of artist relationships.

In [2]:
logger.info('Loading images')

images = defaultdict(list)

for path in tqdm_notebook(glob.glob('tracks/**/*.png', recursive=True)):
    mbid = os.path.basename(path).rsplit('-', 1)[0]
    img = np.rot90(np.array(Image.open(path).convert('L')) / 255)
    #img = np.reshape(img, img.shape + (1,))
    images[mbid].append(img)
    
logger.info('Loaded images')

Loading images


Loaded images


In [3]:
logger.info('Loading dev set')

raw_dataset = pd.read_hdf('dataset/dev_min.hd5', key='artists')

logger.info('Loaded dev set')

Loading dev set
Loaded dev set


In [4]:
logger.info('Artists in images: {}'.format(len(images)))
logger.info('Dataset size: {}'.format(raw_dataset.shape))

Artists in images: 975
Dataset size: (49, 49)


## Define accuracy metric

For my accuracy metric, I get the top similar artists for each artist from the original dataset, then to calculate accuracy of the model, I use the model to calculate all pairwise similarity scores and get the top predicted similar artists. Then I compare the two using edit distance, and return the average value as an accuracy.

Calculating accuracy like this is slow because you pretty much have to go through a full epoch. It's probably only useful at the end of training.

There are two accuracy metrics I use, all of which mean slightly different things:

`compare_accuracy_edit_dist` compares the top n artists in each dataframe by edit distance, and returns it as a proportion to the maximum edit distance possible (i.e. n)

`compare_accuracy_unordered` compares how many of the top n artists are shared between both dataframes, disregarding order, as a proportion of n. 

Additionally, here I define a baseline which simply takes the L2 norm of the difference between the two tracks, averaging to get the similarity score. This typically performs better than random.

In [24]:
def similarity_matrix_to_top(matrix):
    # Converts a similarity score matrix to a sorted table of most similar artists
    most_similar = np.argsort(-matrix.values, axis=1)
    return pd.DataFrame(most_similar, index=matrix.index).T

def eval_artist_similarity(artist_A, artist_B, model):
    # Collect all pairwise combinations of artist tracks, evaluates the model on them, then
    # returns the similarity as an average of all the predictions
    audio_A = images[artist_A]
    audio_B = images[artist_B]
    
    X_tmp = [list(), list()]
    for pair in itertools.product(audio_A, audio_B):
        X_tmp[0].append(pair[0])
        X_tmp[1].append(pair[1])
    
    return np.mean(model.predict(X_tmp))

def eval_artist_baseline(artist_A, artist_B):
    # Similar to above, but using baseline evaluation of taking L2 distance between
    # spectrograms and averaging
    audio_A = images[artist_A]
    audio_B = images[artist_B]
    
    dist = []
    for pair in itertools.product(audio_A, audio_B):
        dist.append(np.linalg.norm(pair[0]-pair[1]))
    
    return -np.mean(np.array(dist))

def create_similarity_matrix(artists, model, all_artists = None, baseline = False):
    # Creates a similarity matrix for a list of artists, using a given model to predict similarity
    if all_artists is None:
        all_artists = artists
    df = pd.DataFrame(np.zeros((len(artists), len(all_artists))), columns = all_artists, index = artists)
    for artist_A, artist_B in itertools.product(all_artists, artists):
        if artist_A != artist_B:
            if baseline:
                sim = eval_artist_baseline(artist_A, artist_B)
            else:
                sim = eval_artist_similarity(artist_A, artist_B, model)
            df[artist_A][artist_B] = sim
    return df

def compare_accuracy_edit_dist(df1, df2, n=10):
    # Compares accuracy of two similarity ranking dataframes using edit distance, comparing the top n ranked
    # similar artists
    distances = dict()
    for column in df1:
        distances[column] = difflib.SequenceMatcher(None, df1.head(n)[column], df2.head(n)[column]).ratio()
    return np.mean(np.array(list(distances.values()))), distances

def compare_accuracy_unordered(df1, df2, n=10):
    # Compares accuracy of two similarity ranking dataframes by looking at the size of the unordered union of sets,
    # comparing the top n ranked similar artists
    distances = dict()
    for column in df1:
        distances[column] = len(set(df1.head(n)[column].values).intersection(df2.head(n)[column].values))\
            / len(df1.head(n)[column])
    return np.mean(np.array(list(distances.values()))), distances

## Split into dev-train and dev-test sets

For better validation, split the dev set 75:25 into a dev-train and dev-test set. As how we generated the original train/test split, we will split based on artist.

In [6]:
logger.info('Splitting dev dataset into dev-train and dev-test sets')

allartists = raw_dataset.index

train_artists, test_artists = train_test_split(allartists, test_size=0.25, random_state = 1)

train_set = raw_dataset.loc[train_artists, train_artists]
test_set = raw_dataset.loc[test_artists]
test_set_ranked = similarity_matrix_to_top(test_set)


Splitting dev dataset into dev-train and dev-test sets


## Create dataset relating image references to values

Previously, we had the large raw dataset which is of the form `raw_dataset[artistA][artistB]=similarity`. Here, we create X and Y arrays, with the X array being a list of two arrays, each containing references to one of the spectrogram images that was loaded for a given artist. The Y array is a list containing the similarity scores for the corresponding two artists.

In [7]:
logger.info('Re-formatting datasets for training')

train_dedup = train_set.where(~np.triu(np.ones(train_set.shape)).astype(np.bool))
train_stacked = train_dedup.stack()

test_dedup = test_set.where(~np.triu(np.ones(test_set.shape)).astype(np.bool))
test_stacked = test_dedup.stack()

def format_dataset(raw_dataset):
    X_1 = []
    X_2 = []
    Y = []
    
    for index, value in tqdm_notebook(raw_dataset.iteritems(), total = len(raw_dataset)):
        artist_A = index[0]
        artist_B = index[1]
        audio_A = images[artist_A]
        audio_B = images[artist_B]

        for pair in itertools.product(audio_A, audio_B):
            X_1.append(pair[0])
            X_2.append(pair[1])
            Y.append(value)

    Y = np.array(Y)
    X = [X_1, X_2]
    return X, Y

X_train, Y_train = format_dataset(train_stacked)
X_test, Y_test = format_dataset(test_stacked)

logger.info('Datasets formatted')

Re-formatting datasets for training


Datasets formatted


## Define Keras model specification

This defines a siamese network, which trains the same model with the same parameters and applies it to both images. The output of this shared vector is a fully-connected network with 512 neurons, for each image. The L1 distance between these two networks is then taken and the resulting 512 length vector is fed into a final sigmoid function.

In [8]:
def L1_distance(x):
    return K.abs(x[0] - x[1])

def L1_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_base_network(input_shape):
    input = Input(shape = input_shape)
    x = ZeroPadding1D()(input) # 258x64
    x = Conv1D(128,3,activation='relu')(x) # 256x128
    x = BatchNormalization()(x)
    x = MaxPooling1D(4)(x) # 64x128
    x = ZeroPadding1D()(x) # 66x128
    x = Conv1D(128,3,activation='relu')(x) # 64x128
    x = BatchNormalization()(x)
    x = MaxPooling1D()(x) # 32x128
    x = ZeroPadding1D()(x) # 34x128
    x = Conv1D(128,3,activation='relu')(x) # 32x128
    x = BatchNormalization()(x) 
    x = GlobalMaxPooling1D()(x) # 128
    x = Dense(128, activation='relu')(x) # 128
    return Model(input, x)
    
def model_spec(lr = 0.001, decay = 0.0, **kwargs):
    input_shape = (256,64)

    siamese_net = create_base_network(input_shape)

    input_a = Input(shape = input_shape)
    input_b = Input(shape = input_shape)

    process_a = siamese_net(input_a)
    process_b = siamese_net(input_b)

    distance = Lambda(L1_distance, output_shape = L1_dist_output_shape)([process_a, process_b])

    output = Dense(1, activation='sigmoid')(distance)

    model = Model([input_a, input_b], output)

    adam = Adam(lr=lr, decay=decay)
    model.compile(loss = 'mean_squared_error', optimizer = adam)
    
    return model

model = model_spec()

## Hyperparameter Search

This function will take in a model and a hyperparameters dict, and output training and test losses after training. The following hyperparameters are implemented:

`batch_size`: batch size for training

`epochs`: number of epochs to use in the training phase

`lr`: learning rate for model

`decay`: decay for learning rate

In [9]:
def batch_loader(X_data, Y_data, batch_size):
    curr_batch = 0
    while True:
        end = min(curr_batch + batch_size, len(Y_data))
        X = [np.asarray(X_data[0][curr_batch:end]), np.asarray(X_data[1][curr_batch:end])]
        Y = np.asarray(Y_data[curr_batch:end])
        yield (X, Y)
        if end == len(Y_data):
            curr_batch = 0
        else:
            curr_batch = end

In [10]:
def build_model_path(hparams):
    path_components = ['_'.join([key, str(val)]) for key,val in hparams.items()]
    return os.path.join('models', *path_components)

def test_hyperparams(hparams, model_spec, log=True, save=True, test_accs = [3, 5, 10, 25]):
    # Make the path to the logger/model directory, named based on parameters
    path = build_model_path(hparams)
    os.makedirs(path, exist_ok=True)
    
    # Get temporary logger with hparams.__str__ name 
    tmp_logger = make_logger(str(hparams), os.path.join(path, 'training.log'), )
    
    batch_size = hparams.get('batch_size', 128)
    epochs = hparams.get('epochs', 25)
    model = model_spec(**hparams)

    tmp_logger.info('Training with params {}'.format(hparams))

    history = model.fit_generator(batch_loader(X_train, Y_train, batch_size), epochs=epochs, 
                                  validation_data=batch_loader(X_test, Y_test, batch_size), 
                                  steps_per_epoch=np.ceil(len(Y_train) / batch_size), 
                                  validation_steps=np.ceil(len(Y_test) / batch_size))
    return model
    sim_matrix = create_similarity_matrix(test_artists, model, all_artists=allartists)
    edit_dist_accuracies = {}
    unordered_accuracies = {}
    
    for acc in test_accs:
        edit_dist_accuracy, _ = compare_accuracy_edit_dist(similarity_matrix_to_top(sim_matrix), test_set_ranked, n=acc)
        unordered_accuracy, _ = compare_accuracy_unordered(similarity_matrix_to_top(sim_matrix), test_set_ranked, n=acc)
        edit_dist_accuracies[acc] = edit_dist_accuracy
        unordered_accuracies[acc] = unordered_accuracy

    tmp_logger.info('Finished training, final train loss = {:.5f}, test loss = {:.5f}'.format(
        history.history['loss'][-1], history.history['val_loss'][-1]))
        
    tmp_logger.info('Edit Distance Accuracy: {}'.format(edit_dist_accuracies))
    tmp_logger.info('Unordered Accuracy: {}'.format(unordered_accuracies))
    
    with open(os.path.join(path, 'history'), 'wb') as file:
        pickle.dump(history.history, file)
        
    with open(os.path.join(path, 'edit_dist_acc.pickle'), 'wb') as file:
        pickle.dump(edit_dist_accuracies, file)
        
    with open(os.path.join(path, 'unordered_acc.pickle'), 'wb') as file:
        pickle.dump(unordered_accuracies, file)
    
    if save:
        tmp_logger.info('Saving model')
        model.save(os.path.join(path, 'post_train_model.hd5'))
        tmp_logger.info('Saved model')
        
    return history.history, edit_dist_accuracies, unordered_accuracies, model
        

## Iterate through Hyperparameter sets and Evaluate

Create a dict containing all values for all hyperparameters we want to look at, then generate all combinations of these parameters and test each one.

In [ ]:
all_hparams = {
    'batch_size': [256, 512, 1024],
    'epochs': [10, 25, 50], 
    'lr': [0.01, 0.001, 0.0001]
}

hparams_list = list()
keys, vals = zip(*all_hparams.items())
for val in itertools.product(*vals):
    hparams_list.append(dict(zip(keys, val)))

logger.info('Generated hyperparameters test list, total of {} combinations'.format(len(hparams_list)))

results = []

for hparams in tqdm_notebook(hparams_list):
    logger.info('Testing hyperparameter set {}'.format(hparams))
    history, edit_dist_accs, unordered_accs = test_hyperparams(hparams, model_spec)
    results.append((hparams, history, edit_dist_accs, unordered_accs))

## Random testing stuff below

Test a specific model, and evaluate accuracy with both the model and the baseline.

In [11]:
hparams = {
    'batch_size': 1024,
    'epochs': 10
}

model = test_hyperparams(hparams, model_spec)

Training with params {'batch_size': 1024, 'epochs': 10}


Epoch 1/10
5/5 [==============================] - 10s 2s/step - loss: 0.0456 - val_loss: 0.0039
Epoch 2/10
5/5 [==============================] - 2s 407ms/step - loss: 0.0027 - val_loss: 0.0039
Epoch 3/10
5/5 [==============================] - 2s 421ms/step - loss: 0.0027 - val_loss: 0.0039
Epoch 4/10
5/5 [==============================] - 2s 416ms/step - loss: 0.0028 - val_loss: 0.0040
Epoch 5/10
5/5 [==============================] - 2s 412ms/step - loss: 0.0028 - val_loss: 0.0040
Epoch 6/10
5/5 [==============================] - 2s 426ms/step - loss: 0.0028 - val_loss: 0.0039
Epoch 7/10
5/5 [==============================] - 2s 404ms/step - loss: 0.0028 - val_loss: 0.0040
Epoch 8/10
5/5 [==============================] - 2s 414ms/step - loss: 0.0028 - val_loss: 0.0040
Epoch 9/10
5/5 [==============================] - 2s 405ms/step - loss: 0.0027 - val_loss: 0.0040
Epoch 10/10
5/5 [==============================] - 2s 413ms/step - loss: 0.0027 - val_loss: 0.0040


In [25]:
sim_matrix = create_similarity_matrix(test_artists, model, all_artists=allartists)
sim_matrix_baseline = create_similarity_matrix(test_artists, model, all_artists=allartists, baseline=True)

In [26]:
sim_ranks = similarity_matrix_to_top(sim_matrix)
sim_baseline_ranks = similarity_matrix_to_top(sim_matrix_baseline)

for acc in [3, 5, 15]:
    unord_model_acc, _ = compare_accuracy_unordered(sim_ranks, test_set_ranked, n=acc)
    unord_base_acc, _ = compare_accuracy_unordered(sim_baseline_ranks, test_set_ranked, n=acc)
    edit_model_acc, _ = compare_accuracy_edit_dist(sim_ranks, test_set_ranked, n=acc)
    edit_base_acc, _ = compare_accuracy_edit_dist(sim_baseline_ranks, test_set_ranked, n=acc)
    
    print('''Unordered accuracy; model: {}, baseline: {}. 
    Edit Distance Accuracy; model: {}, baseline: {}, Random unordered accuracy: {}'''.format(
        unord_model_acc,
        unord_base_acc,
        edit_model_acc,
        edit_base_acc,
        acc / len(allartists)
    ))

Unordered accuracy; model: 0.10256410256410256, baseline: 0.10256410256410256. 
    Edit Distance Accuracy; model: 0.10256410256410256, baseline: 0.10256410256410256, Random unordered accuracy: 0.061224489795918366
Unordered accuracy; model: 0.24615384615384622, baseline: 0.15384615384615385. 
    Edit Distance Accuracy; model: 0.18461538461538465, baseline: 0.13846153846153844, Random unordered accuracy: 0.10204081632653061
Unordered accuracy; model: 0.38974358974358975, baseline: 0.34358974358974365. 
    Edit Distance Accuracy; model: 0.1794871794871795, baseline: 0.17948717948717946, Random unordered accuracy: 0.30612244897959184
